In [7]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re

In [4]:
# science
urls = []
# url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
url = 'https://search.sciencemag.org/?searchTerm=kaist&order=newest&limit=textFields&pageSize=100&startDate=2015-01-01&&'
browser = webdriver.Chrome('../../chromedriver_win32/chromedriver.exe')
browser.get(url)
timeout = 30
try:
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'media__headline'))
    WebDriverWait(browser, timeout).until(element_present)
except Exception as e :
    print(e)
    print("Timed out waiting for page to load")
html = browser.page_source
soup = bs(html, "html.parser")
elements = soup.select('.media__headline a[href]')
for element in elements:
    link = element.attrs['href']
    urls.append(link)
browser.quit()

print('Whole post num : {}'.format(len(urls)))


Whole post num : 52


In [8]:
titles = []
contents = []
for i,url in enumerate(urls):
    response = requests.get(url)
    html = response.text
    soup = bs(html,'html.parser')
    try: 
        response = requests.get(url)    
        html = response.text
        soup = bs(html,'html.parser')
        content = soup.select('.abstract p')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
        content = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', content)
        title = soup.select('.highwire-cite-title')[0].get_text()
    except Exception as e: 
        print(content,title)
    contents.append(content)
    titles.append(title)
    if i%10 == 0:
        print(i)


0
10
20
30
Highprecision monitoring of electrophysiological signals with high spatial and temporal resolutions is one of the most important subjects for elucidating physiology functions Recently ultraflexible multielectrode arrays MEAs have been fabricated to establish conformal contacts with the surface of organs and to measure propagation of electrophysiological signals with high spatialtemporal resolution; however plastic substrates have high Young’s modulus causing difficulties in creating appropriate stretchability and blood compatibility for applying them on the dynamically moving and surgical bleeding surface of the heart Here we have successfully fabricated an active MEA that simultaneously achieves nonthrombogenicity stretchability and stability which allows longterm electrocardiographic ECG monitoring of the dynamically moving hearts of rats even with capillary bleeding Because of the active data readout the measured ECG signals exhibit a high signaltonoise ratio of 52 dB The

In [16]:
data = {"title" : titles,"content": contents,"url": urls}
df = pd.DataFrame(data)
df['content'] =df['content'].map(lambda x : x.replace('\r',""))
df.to_pickle('science_post.pkl')
df.to_csv('kaist_science.csv',encoding='utf-8-sig')